In [8]:
# 🔄 Section 1: Full Extraction
import pandas as pd

df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print(f"Extracted {len(df_full)} rows fully.")
print("Columns:", df_full.columns.tolist())
df_full.head()


Extracted 212 rows fully.
Columns: ['order_id', 'customer', 'amount', 'date', 'last_updated']


,order_id,customer,amount,date,last_updated
0,2018,Costco,938,2025-04-01,2025-04-01 05:00:00
1,9235,Costco,1436,2025-04-01,2025-04-01 02:26:00
2,6736,Amazon,956,2025-04-01,2025-04-01 07:22:00
3,5108,Costco,1243,2025-04-01,2025-04-01 08:45:00
4,9094,Costco,1114,2025-04-01,2025-04-01 17:46:00


In [9]:
import shutil

# Use a raw string (r"") to prevent backslash issues
source_path = r"C:\Users\PC\OneDrive\Desktop\ETL_Extract_Paul Mbuvi_669984\custom_data.csv"

# This moves the file to the same folder as your notebook
shutil.move(source_path, 'custom_data.csv')

'custom_data.csv'

In [10]:
# ⏩ Section 2: Incremental Extraction
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

last_time = pd.to_datetime(last_extraction)
df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
df_incremental = df[df["last_updated"] > last_time]

print(f"Extracted {len(df_incremental)} rows incrementally since {last_extraction}.")
df_incremental.head()


Extracted 0 rows incrementally since 2025-05-30T18:00:00.


,order_id,customer,amount,date,last_updated


In [11]:
# 💾 Section 3: Save New Timestamp
new_checkpoint = df['last_updated'].max()
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())

print(f"Updated last_extraction.txt to {new_checkpoint}")


Updated last_extraction.txt to 2025-05-30 18:00:00


In [12]:
import pandas as pd

df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print("Column Names:", df.columns.tolist())
df.head()




Column Names: ['order_id', 'customer', 'amount', 'date', 'last_updated']


,order_id,customer,amount,date,last_updated
0,2018,Costco,938,2025-04-01,2025-04-01 05:00:00
1,9235,Costco,1436,2025-04-01,2025-04-01 02:26:00
2,6736,Amazon,956,2025-04-01,2025-04-01 07:22:00
3,5108,Costco,1243,2025-04-01,2025-04-01 08:45:00
4,9094,Costco,1114,2025-04-01,2025-04-01 17:46:00


In [13]:
# 1. Copy original full dataset
df_full_cleaned = df_full.copy()

# 2. Clean missing values in 'amount' (if any)
df_full_cleaned['amount'] = df_full_cleaned['amount'].fillna(df_full_cleaned['amount'].median())

# 3. Enrich: Add a new column with tax (10%)
df_full_cleaned['amount_with_tax'] = df_full_cleaned['amount'] * 1.10

# 4. Structure: Standardize date format
df_full_cleaned['date'] = pd.to_datetime(df_full_cleaned['date']).dt.strftime('%Y-%m-%d')

# 5. Categorize: Bin amount into levels
bins = [0, 1000, 2000, float('inf')]
labels = ['Low', 'Medium', 'High']
df_full_cleaned['amount_level'] = pd.cut(df_full_cleaned['amount'], bins=bins, labels=labels)

# 6. Save transformed full dataset
df_full_cleaned.to_csv("transformed_full.csv", index=False)

print("✅ Full data transformed and saved to transformed_full.csv")
df_full_cleaned.head()


✅ Full data transformed and saved to transformed_full.csv


,order_id,customer,amount,date,last_updated,amount_with_tax,amount_level
0,2018,Costco,938,2025-04-01,2025-04-01 05:00:00,1031.8,Low
1,9235,Costco,1436,2025-04-01,2025-04-01 02:26:00,1579.6,Medium
2,6736,Amazon,956,2025-04-01,2025-04-01 07:22:00,1051.6,Low
3,5108,Costco,1243,2025-04-01,2025-04-01 08:45:00,1367.3,Medium
4,9094,Costco,1114,2025-04-01,2025-04-01 17:46:00,1225.4,Medium


In [14]:
# 1. Clean copy of incremental data
df_incremental_cleaned = df_incremental.copy()

# 2. Clean missing 'amount'
df_incremental_cleaned['amount'] = df_incremental_cleaned['amount'].fillna(df_incremental_cleaned['amount'].median())

# 3. Add tax
df_incremental_cleaned['amount_with_tax'] = df_incremental_cleaned['amount'] * 1.10

# 4. Format date
df_incremental_cleaned['date'] = pd.to_datetime(df_incremental_cleaned['date']).dt.strftime('%Y-%m-%d')

# 5. Categorize amount
df_incremental_cleaned['amount_level'] = pd.cut(df_incremental_cleaned['amount'], bins=bins, labels=labels)

# 6. Save
df_incremental_cleaned.to_csv("transformed_incremental.csv", index=False)

print("✅ Incremental data transformed and saved to transformed_incremental.csv")
df_incremental_cleaned.head()


✅ Incremental data transformed and saved to transformed_incremental.csv


,order_id,customer,amount,date,last_updated,amount_with_tax,amount_level
